建立基础回测框架

In [46]:
%matplotlib inline

matplotlib.use('TkAgg')  # 尝试更改为常用后端

In [47]:
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import backtrader as bt
import pandas as pd
from self_package import get_stock
from self_package import tech_index as ti

In [48]:
df_ts = get_stock.get_stock_data("601111", "daily", "20000101", "20250407")
df_ts = get_stock.standardize_stock_data(df_ts)



In [15]:
# 定义策略
class SMAStrategy(bt.Strategy):

    def __init__(self):
        self.dataclose = self.data0.close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        self.sma = bt.indicators.SimpleMovingAverage(self.data0, period=15)
    
    def next(self):
        if not self.position:
            if self.dataclose[0] > self.sma[0]:
                self.buy()
        else:
            if self.dataclose[0] < self.sma[0]:
                self.close()

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price, 
                     order.executed.value, 
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log("SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f" %
                    (order.executed.price, 
                     order.executed.value, 
                     order.executed.comm))
                self.bar_executed = len(self)
        elif order.status in [order.Margin, order.Rejected]:
            self.log("Order failed")
        self.order = None

    def log(self, txt, dt=None,doprint=True):
        if doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

In [49]:
df_ts

,Stock_Code,Open,Close,High,Low,Volume,Turnover,Amplitude,Price_Change_Pct,Price_Change,Turnover_Rate
Date,,,,,,,,,,,
2006-08-18,601111,2.78,2.80,2.81,2.74,2850715,796088312.0,2.50,0.00,0.00,34.79
2006-08-21,601111,2.80,2.80,2.80,2.75,516381,143513288.0,1.79,0.00,0.00,6.30
2006-08-23,601111,2.78,2.78,2.79,2.77,251275,69931225.0,0.71,-0.71,-0.02,3.07
2006-08-24,601111,2.77,2.78,2.79,2.75,288786,80010152.0,1.44,0.00,0.00,3.52
2006-08-25,601111,2.77,2.76,2.78,2.75,163153,45118024.0,1.08,-0.72,-0.02,1.99
...,...,...,...,...,...,...,...,...,...,...,...
2025-03-31,601111,7.28,7.12,7.30,7.10,768348,551722519.0,2.74,-2.47,-0.18,0.66
2025-04-01,601111,7.14,7.15,7.21,7.05,627694,447537774.0,2.25,0.42,0.03,0.54
2025-04-02,601111,7.14,7.13,7.20,7.10,359331,256452643.0,1.40,-0.28,-0.02,0.31


In [50]:
if  __name__ == "__main__":
    matplotlib.get_backend()

    cerebro = bt.Cerebro()

    # 读取数据
    stock_value = df_ts
    

    # 读取数据
    cerebro.adddata(data, name="601857")
    
    # 读取策略 这一步是关键 
    cerebro.addstrategy(SMAStrategy)

    # 计算指标
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name = "sharpe")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name = "drawdown")

    # 设置初始资金
    cerebro.broker.setcash(10000)
    # 设置佣金
    cerebro.broker.setcommission(commission=0.0006)
    # 设置购买比例
    cerebro.addsizer(bt.sizers.PercentSizer, percents=30 )

    # 开始测试
    result = cerebro.run()
    
    # 修改这行
    print("夏普比率分析结果:", result[0].analyzers.sharpe.get_analysis()['sharperatio'])
    print("最大回撤分析结果:", result[0].analyzers.drawdown.get_analysis()['drawdown'])

    # ----------------------
    # 绘图部分 
    # ----------------------

    # 绘制结果
    figs = cerebro.plot(style='candle',
                   barup='red', bardown='green',
                   volup='red', voldown='green',
                   grid=True, volume=True,
                   iplot=False)  # iplot=False 对某些环境很重要
                    
    # 然后显示图表
    plt.show()



2006-09-11, BUY EXECUTED, Price: 2.86, Cost: 3000.00, Comm 1.80
2006-10-18, SELL EXECUTED, Price: 3.47, Cost: 3000.00, Comm 2.18
2006-10-20, BUY EXECUTED, Price: 3.58, Cost: 3173.04, Comm 1.90
2006-10-23, SELL EXECUTED, Price: 3.55, Cost: 3173.04, Comm 1.89
2006-10-24, BUY EXECUTED, Price: 3.59, Cost: 3199.47, Comm 1.92
2006-11-27, SELL EXECUTED, Price: 4.36, Cost: 3199.47, Comm 2.33
2006-11-28, BUY EXECUTED, Price: 4.41, Cost: 3378.58, Comm 2.03
2006-11-29, SELL EXECUTED, Price: 4.28, Cost: 3378.58, Comm 1.97
2006-11-30, BUY EXECUTED, Price: 4.42, Cost: 3355.17, Comm 2.01
2006-12-11, SELL EXECUTED, Price: 4.47, Cost: 3355.17, Comm 2.04
2006-12-12, BUY EXECUTED, Price: 4.71, Cost: 3386.91, Comm 2.03
2006-12-19, SELL EXECUTED, Price: 4.66, Cost: 3386.91, Comm 2.01
2006-12-20, BUY EXECUTED, Price: 4.75, Cost: 3318.41, Comm 1.99
2006-12-21, SELL EXECUTED, Price: 4.65, Cost: 3318.41, Comm 1.95
2006-12-29, BUY EXECUTED, Price: 4.84, Cost: 3445.09, Comm 2.07
2007-02-01, SELL EXECUTED, Price: